### `pke` package: https://github.com/boudinfl/pke

```bash
pip install git+https://github.com/boudinfl/pke.git

python -m spacy download en_core_web_sm
```

### Models 
* TopicalPageRank (Sterckx et al., 2015)
    - Sterckx, Lucas, Thomas Demeester, Johannes Deleu, and Chris Develder. "Topical word importance for fast keyphrase extraction." In Proceedings of the 24th International Conference on World Wide Web, pp. 121-122. 2015.
* PositionRank (Florescu and Caragea, 2017)
    - Florescu, Corina, and Cornelia Caragea. "Positionrank: An unsupervised approach to keyphrase extraction from scholarly documents." In Proceedings of the 55th Annual Meeting of the Association for Computational Linguistics (Volume 1: Long Papers), pp. 1105-1115. 2017.
* MultipartiteRank (Boudin, 2018)
    - [Unsupervised Keyphrase Extraction with Multipartite Graphs (Boudin, NAACL 2018)](https://aclanthology.org/N18-2105/)

In [1]:
import pke
import os
import pandas as pd
from tqdm import tqdm

C:\Users\Jihye Park\anaconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
filepath = 'data/preprocessed_sentences_for_each_period_1997Q1-2019Q3.csv'

save_dir = 'C:\DATA\hot_topic_detection_in_central_bankers_speeches'
save_filepath_format = os.path.join(save_dir, 'top5_hot_topics_{}.csv') # model name

In [3]:
df = pd.read_csv(filepath)
sorted_periods = sorted(df[df['period'].apply(lambda x: x[:4]!='1997')].period.unique())
df.set_index('period', inplace=True)
print(sorted_periods)
df.head()

['1998_Q1', '1998_Q2', '1998_Q3', '1998_Q4', '1999_Q1', '1999_Q2', '1999_Q3', '1999_Q4', '2000_Q1', '2000_Q2', '2000_Q3', '2000_Q4', '2001_Q1', '2001_Q2', '2001_Q3', '2001_Q4', '2002_Q1', '2002_Q2', '2002_Q3', '2002_Q4', '2003_Q1', '2003_Q2', '2003_Q3', '2003_Q4', '2004_Q1', '2004_Q2', '2004_Q3', '2004_Q4', '2005_Q1', '2005_Q2', '2005_Q3', '2005_Q4', '2006_Q1', '2006_Q2', '2006_Q3', '2006_Q4', '2007_Q1', '2007_Q2', '2007_Q3', '2007_Q4', '2008_Q1', '2008_Q2', '2008_Q3', '2008_Q4', '2009_Q1', '2009_Q2', '2009_Q3', '2009_Q4', '2010_Q1', '2010_Q2', '2010_Q3', '2010_Q4', '2011_Q1', '2011_Q2', '2011_Q3', '2011_Q4', '2012_Q1', '2012_Q2', '2012_Q3', '2012_Q4', '2013_Q1', '2013_Q2', '2013_Q3', '2013_Q4', '2014_Q1', '2014_Q2', '2014_Q3', '2014_Q4', '2015_Q1', '2015_Q2', '2015_Q3', '2015_Q4', '2016_Q1', '2016_Q2', '2016_Q3', '2016_Q4', '2017_Q1', '2017_Q2', '2017_Q3', '2017_Q4', '2018_Q1', '2018_Q2', '2018_Q3', '2018_Q4', '2019_Q1', '2019_Q2', '2019_Q3']


,document
period,
1997_Q1,Mr. Greenspan addresses some key roles of a ce...
1997_Q2,Mr. Greenspan highlights some key aspects of t...
1997_Q3,Mr. Greenspan presents the views of the Federa...
1997_Q4,Mr. Greenspan considers some of the effects of...
1998_Q1,Mr. Greenspan’s remarks to the American Econom...


In [4]:
def get_keyphrases(extractor, doc, n=5):
    extractor.load_document(input=doc, language='en')

    extractor.candidate_selection()
    extractor.candidate_weighting()

    keyphrases = extractor.get_n_best(n=n)
    return keyphrases # (keyphrase, score) tuples

### TopicalPageRank (Sterckx et al., 2015)
Sterckx, Lucas, Thomas Demeester, Johannes Deleu, and Chris Develder. "Topical word importance for fast keyphrase extraction." In Proceedings of the 24th International Conference on World Wide Web, pp. 121-122. 2015.

In [11]:
extractor = pke.unsupervised.TopicalPageRank()
save_filepath = save_filepath_format.format('TopicalPageRank')

In [12]:
dfs = []
for _, period in tqdm(enumerate(sorted_periods)):
    doc = df.loc[period]['document']
    keywords = get_keyphrases(extractor, doc, n=5)
    one_period_df = pd.DataFrame(keywords, columns=['word', 'score'])
    
    one_period_df['period'] = period
    dfs.append(one_period_df)
    
top5_df = pd.concat(dfs)

top5_df.to_csv(save_filepath, index=False)
print('Created {}'.format(save_filepath))

# Key periods
top5_df[(top5_df['period'].isin(['1998_Q2', '2000_Q1', '2007_Q2']))]

0it [00:00, ?it/s]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
C:\Users\Jihye Park\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
1it [00:14, 14.41s/it]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
C:\Users\Jihye Park\anaconda3\lib\site-packages\sklearn\feature_extraction\text.p

11it [02:08, 11.51s/it]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
C:\Users\Jihye Park\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
12it [02:18, 10.84s/it]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
C:\Users\Jihye Park\anaconda3\lib\site-packages\sklearn\feature_extraction\

22it [04:18, 11.52s/it]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
C:\Users\Jihye Park\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
23it [04:25, 10.24s/it]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
C:\Users\Jihye Park\anaconda3\lib\site-packages\sklearn\feature_extraction\

33it [06:52, 14.16s/it]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
C:\Users\Jihye Park\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
34it [07:08, 14.66s/it]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
C:\Users\Jihye Park\anaconda3\lib\site-packages\sklearn\feature_extraction\

44it [09:55, 16.57s/it]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
C:\Users\Jihye Park\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
45it [10:13, 16.75s/it]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
C:\Users\Jihye Park\anaconda3\lib\site-packages\sklearn\feature_extraction\

55it [12:59, 16.15s/it]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
C:\Users\Jihye Park\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
56it [13:09, 14.31s/it]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
C:\Users\Jihye Park\anaconda3\lib\site-packages\sklearn\feature_extraction\

66it [15:16, 12.23s/it]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
C:\Users\Jihye Park\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
67it [15:27, 11.88s/it]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
C:\Users\Jihye Park\anaconda3\lib\site-packages\sklearn\feature_extraction\

77it [17:17, 11.74s/it]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
C:\Users\Jihye Park\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
78it [17:26, 10.77s/it]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
C:\Users\Jihye Park\anaconda3\lib\site-packages\sklearn\feature_extraction\

Created C:\DATA\hot_topic_detection_in_central_bankers_speeches\top5_hot_topics_TopicalPageRank.csv


,word,score,period
0,new high-tech international financial system,0.042367,1998_Q2
1,new international financial system,0.041111,1998_Q2
2,many financial markets,0.038286,1998_Q2
3,international financial system remarks,0.037287,1998_Q2
4,major financial market disruptions,0.036600,1998_Q2
0,many new technologies,0.024574,2000_Q1
1,new financial products,0.023078,2000_Q1
2,productive capital investments,0.022437,2000_Q1
3,innovative financial products,0.022254,2000_Q1
4,competitive market economy,0.021175,2000_Q1


### PositionRank (Florescu and Caragea, 2017)
Florescu, Corina, and Cornelia Caragea. "Positionrank: An unsupervised approach to keyphrase extraction from scholarly documents." In Proceedings of the 55th Annual Meeting of the Association for Computational Linguistics (Volume 1: Long Papers), pp. 1105-1115. 2017.

In [13]:
extractor = pke.unsupervised.PositionRank()
save_filepath = save_filepath_format.format('PositionRank')

In [14]:
dfs = []
for _, period in tqdm(enumerate(sorted_periods)):
    doc = df.loc[period]['document']
    keywords = get_keyphrases(extractor, doc, n=5)
    one_period_df = pd.DataFrame(keywords, columns=['word', 'score'])
    
    one_period_df['period'] = period
    dfs.append(one_period_df)
    
top5_df = pd.concat(dfs)

top5_df.to_csv(save_filepath, index=False)
print('Created {}'.format(save_filepath))

# Key periods
top5_df[(top5_df['period'].isin(['1998_Q2', '2000_Q1', '2007_Q2']))]

0it [00:00, ?it/s]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
1it [00:09,  9.60s/it]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
2it [00:16,  8.06s/it]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
3it [00:25,  8.61s/it]C:\Users\Jihye Park\anaconda3\lib\site-pack

27it [03:30,  8.45s/it]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
28it [03:41,  9.19s/it]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
29it [03:55, 10.52s/it]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
30it [04:09, 11.56s/it]C:\Users\Jihye Park\anaconda3\lib\s

54it [09:18, 13.62s/it]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
55it [09:27, 12.29s/it]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
56it [09:34, 10.49s/it]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
57it [09:44, 10.58s/it]C:\Users\Jihye Park\anaconda3\lib\s

81it [13:06,  7.66s/it]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
82it [13:11,  6.89s/it]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
83it [13:17,  6.51s/it]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
84it [13:24,  6.78s/it]C:\Users\Jihye Park\anaconda3\lib\s

Created C:\DATA\hot_topic_detection_in_central_bankers_speeches\top5_hot_topics_PositionRank.csv


,word,score,period
0,mr. alan greenspan,0.069165,1998_Q2
1,mr. greenspan,0.062879,1998_Q2
2,free market systems,0.048051,1998_Q2
3,market capitalism remarks,0.046764,1998_Q2
4,mr. chairman,0.046381,1998_Q2
0,mr alan greenspan,0.074618,2000_Q1
1,mr greenspan,0.066422,2000_Q1
2,many new technologies,0.038920,2000_Q1
3,new technologies,0.033363,2000_Q1
4,us economy remarks,0.033355,2000_Q1


### MultipartiteRank (Boudin, 2018)
[Unsupervised Keyphrase Extraction with Multipartite Graphs (Boudin, NAACL 2018)](https://aclanthology.org/N18-2105/)

In [15]:
extractor = pke.unsupervised.MultipartiteRank()
save_filepath = save_filepath_format.format('MultipartiteRank')

In [17]:
dfs = []
for _, period in tqdm(enumerate(sorted_periods)):
    doc = df.loc[period]['document']
    keywords = get_keyphrases(extractor, doc, n=5)
    one_period_df = pd.DataFrame(keywords, columns=['word', 'score'])
    
    one_period_df['period'] = period
    dfs.append(one_period_df)
    
top5_df = pd.concat(dfs)

top5_df.to_csv(save_filepath, index=False)
print('Created {}'.format(save_filepath))

# Key periods
top5_df[(top5_df['period'].isin(['1998_Q2', '2000_Q1', '2007_Q2']))]

1it [05:05, 305.45s/it]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
2it [07:49, 222.45s/it]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
3it [11:46, 228.86s/it]C:\Users\Jihye Park\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
3it [11:59, 239.92s/it]


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Jihye Park\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Jihye Park\AppData\Local\Temp\ipykernel_21832\3222944803.py", line 4, in <module>
    keywords = get_keyphrases(extractor, doc, n=5)
  File "C:\Users\Jihye Park\AppData\Local\Temp\ipykernel_21832\3198141450.py", line 5, in get_keyphrases
    extractor.candidate_weighting()
  File "C:\Users\Jihye Park\anaconda3\lib\site-packages\pke\unsupervised\graph_based\multipartiterank.py", line 231, in candidate_weighting
    self.build_topic_graph()
  File "C:\Users\Jihye Park\anaconda3\lib\site-packages\pke\unsupervised\graph_based\multipartiterank.py", line 154, in build_topic_graph
    weights.append(1.0 / gap)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Jihye Park\anaconda3\lib\site